In [ ]:
import numpy as np
import pandas as pd

chem_assess = pd.read_excel('data/OpenFoodToxTX22761_2021.xlsx', 'CHEM_ASSESS')
component = pd.read_excel('data/OpenFoodToxTX22761_2021.xlsx', 'COMPONENT')
study = pd.read_excel('data/OpenFoodToxTX22761_2021.xlsx', 'STUDY')
genotox = pd.read_excel('data/OpenFoodToxTX22761_2021.xlsx', 'GENOTOX')

component.set_index('COM_ID')


,SUBSTANCECOMPONENT_ID,SUB_COM_ID,SUB_ID,TRX_ID,SUB_NAME,SUB_ECSUBINVENTENTRYREF,SUB_CASNUMBER,SUB_DESCRIPTION,SUBPARAM_ID,SUBPARAM_CODE,...,COMPARAM_ID,COMPARAM_CODE,COMPARAMNAME,MOLECULARFORMULA,SMILESNOTATION,INCHI,COM_TYPE,COM_STRUCTURESHOWN,SMILESNOTATIONSOURCE,INCHI_NOTATIONSOURCE
COM_ID,,,,,,,,,,,,,,,,,,,,,
486,414401,1,1,41271,Vanadium (total),231-171-1,7440-62-2,Vanadium is widely distributed in the earth's ...,31,RF-00000199-CHE,...,24552,RF-00009930-PAR,Vanadate,NaN,NaN,NaN,inorganic,NaN,NaN,NaN
487,414402,2,1,41271,Vanadium (total),231-171-1,7440-62-2,Vanadium is widely distributed in the earth's ...,31,RF-00000199-CHE,...,24645,RF-00010023-PAR,Sodium vanadate,NaO3V,NaN,NaN,inorganic,NaN,NaN,NaN
488,414403,3,1,41271,Vanadium (total),231-171-1,7440-62-2,Vanadium is widely distributed in the earth's ...,31,RF-00000199-CHE,...,24646,RF-00010024-PAR,Vanadyl sulfate,NaN,NaN,NaN,inorganic,NaN,NaN,NaN
489,414404,4,1,41271,Vanadium (total),231-171-1,7440-62-2,Vanadium is widely distributed in the earth's ...,31,RF-00000199-CHE,...,24647,RF-00010025-PAR,Vanadium pentoxide,O5V2,NaN,NaN,inorganic,NaN,NaN,NaN
490,414405,5,2,41271,Boron compounds,NaN,NaN,Boron is a naturally occurring element that is...,24982,RF-00010360-PAR,...,14270,RF-00000056-ADD,Boric acid,BH3O3,B(O)(O)O,InChI=1S/BH3O3/c2-1(3)4/h2-4H,inorganic,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6916,421785,8063,5462,41271,Vitamin B12 by Ensifer adhaerens CNCM-I 5541,NaN,NaN,The production strain has been deposited at th...,26948,RF-00012308-PAR,...,14189,RF-00000424-NTR,vitamin B-12,C63H88CoN14O14P,[Co++].C[C@H](CN=C(O)CC[C@]1(C)[C@@H](CC(O)=N)...,InChI=1S/C62H90N13O14P.CN.Co/c1-29-20-39-40(21...,organic,NaN,NaN,NaN
994,421786,8064,5463,41271,L-lysine monohydrochloride produced by Coryneb...,NaN,NaN,NaN,26949,RF-00012309-PAR,...,24114,RF-00009493-PAR,L-Lysine hydrochloride,C6H15ClN2O2,C(CCN)CC(C(=O)O)N.Cl,InChI=1S/C6H14N2O2.ClH/c7-4-2-1-3-5(8)6(9)10;/...,organic,NaN,NaN,NaN
587,421787,8065,5464,41271,L-lysine (concentrated liquid base) produced b...,NaN,NaN,NaN,26950,RF-00012310-PAR,...,14088,RF-00000327-NTR,L-lysine,C6H14N2O2,C(CCN)C[C@@H](C(=O)O)N,"InChI=1S/C6H14N2O2/c7-4-2-1-3-5(8)6(9)10/h5H,1...",organic,NaN,NaN,NaN


In [86]:
df = study.merge(component, on='SUB_COM_ID')
df = df[df['SMILESNOTATION'].notna()]
df = df.merge(genotox, on='GENOTOX_ID', how='outer')
df = df.merge(chem_assess, on='HAZARD_ID')

/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'TRX_ID_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


In [ ]:
# df[df.SMILESNOTATION == 'B(O)(O)O'][['SMILESNOTATION', 'RISKVALUE_MILLI', 'SAFETY_FACTOR']].head(20)
# df.groupby('SMILESNOTATION').first().head()
import scipy.stats

df['RISKVAL'] = (df.fillna(1)['SAFETY_FACTOR'] * df['RISKVALUE_MILLI'])
structure_risk = df[['SMILESNOTATION', 'RISKVAL']].groupby('SMILESNOTATION').agg([np.mean, np.std, scipy.stats.sem ])
# structure_risk[structure_risk.SMILESNOTATION == 'B(O)(O)O']['RISKVAL'].dropna(subset=['mean']).plot(kind='barh', y='mean')
# len(structure_risk['RISKVAL'].dropna(subset=['mean']).fillna(0))
len(structure_risk['RISKVAL'].fillna(0))


/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:216: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = umr_sum(x, axis, dtype, out, keepdims)
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  um.multiply(xv, xv, out=xv)


,mean,std,sem
SMILESNOTATION,,,
B(=O)OB([O-])OB([O-])OB=O.[Na+].[Na+],0.160000,0.000000,0.000000
B(O)(O)O,7.427692,3.309678,0.917940
B([O-])([O-])[O-],7.246667,3.388959,0.978308
Brc1cc(cc(Br)c1OC(=O)CCCCCCC)C#N,2.100000,2.078461,1.039230
C#CCN1C(=O)COC2=CC(=C(C=C21)N3C(=O)C4=C(C3=O)CCCC4)F,3.428571,2.940360,1.111351
...,...,...,...
n(ccnc1CCS)c1,0.001500,0.000000,0.000000
n1(c(c(c(=O)n1C(C)C)c1ccccc1C)N)C(=O)SCC=C,12.091667,10.935468,3.156798
n1c(c(c(n2ncnc12)N)CCCCCCCC)CC,1000.000000,0.000000,0.000000
